In [4]:
# project imports
import env
from wayback_search import POLICY_DIR
from build_master_index import MASTER_CSV, PROBLEM_COMPANIES


# python & package imports
import pandas as pd
import os


# plot things
%matplotlib inline
# autoreload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
df = pd.read_csv(MASTER_CSV)
df.policy_date = pd.to_datetime(df.policy_date)
#df['policy_year'] = df.policy_date.dt.year
#df['policy_month'] = df.policy_date.dt.year
#df['policy_day'] = df.policy_date.dt.year
df.head()

,company,policy_date,policy_url,policy_path
0,1password,2017-02-02,https://1password.com/legal/privacy/privacy-20...,1password/1password-2017-02-02.txt
1,1password,2017-03-16,https://1password.com/legal/privacy/privacy-20...,1password/1password-2017-03-16.txt
2,1password,2017-08-02,https://1password.com/legal/privacy/privacy-20...,1password/1password-2017-08-02.txt
3,1password,2018-03-15,https://1password.com/legal/privacy/privacy-20...,1password/1password-2018-03-15.txt
4,1password,2018-04-04,https://1password.com/legal/privacy/privacy-20...,1password/1password-2018-04-04.txt


In [7]:
pd.Series([os.path.exists(os.path.join(POLICY_DIR, x)) for x in df.policy_path])

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
20     True
21     True
22     True
23     True
24     True
25     True
26     True
27     True
28     True
29     True
       ... 
209    True
210    True
211    True
212    True
213    True
214    True
215    True
216    True
217    True
218    True
219    True
220    True
221    True
222    True
223    True
224    True
225    True
226    True
227    True
228    True
229    True
230    True
231    True
232    True
233    True
234    True
235    True
236    True
237    True
238    True
Length: 239, dtype: bool

In [6]:
df[df.policy_path[~pd.Series([os.path.exists(os.path.join(POLICY_DIR, x)) for x in df.policy_path])]]

KeyError: "None of [Index(['netflix/netflix-2007-01-09.txt'], dtype='object')] are in the [columns]"

In [ ]:
print(len(df.company.unique()))
print(len(df.policy_path.unique()))
print(len(df))

In [ ]:
# sometimes duplicate csv rows appear because of overlap in the
# dates of two configurations; they are the same policy most likely
# so we just drop the duplicate
df = df[~df.duplicated('policy_path')]
len(df)

In [ ]:
# some companies proved more difficult than others when gathering
# privacy policies, so we drop them here (the policies are left
# as part of the original dataset because they are valid policies,
# but we do not have all the revisions)
df = df[~df.company.str.contains('|'.join(PROBLEM_COMPANIES))]
len(df)

In [ ]:
df.company.unique()

In [ ]:
print(min(df.policy_date))
print(max(df.policy_date))

In [ ]:
grouped = df.groupby('company').policy_path.count()
grouped.plot.bar(title='Privacy Policy Revisions by Company')

In [ ]:
grouped = df.groupby([df.policy_date.dt.year]).policy_path.count()
grouped.plot.bar(title='Privacy Policy Revisions by Year (All Companies)')

## Google

In [ ]:
google = df[df.company == 'google']
google.head()

In [ ]:
def get_policy_texts(policy_paths):
    """
    Creates a pd.Series of full text from policies

    Args:
        policy_paths: pd.Series, paths to policy txt files
        
    Returns:
        pd.Series, strings
    """
    texts = list()
    for p in policy_paths:
        with open(os.path.join(POLICY_DIR, p), 'r', encoding='utf-8') as f:
            page = f.read()
            texts.append(page)
    return pd.Series(texts)

In [ ]:
get_policy_texts(df.policy_path)